# Setup

In [1]:
!git clone https://github.com/jmerullo/lm_vector_arithmetic.git

Cloning into 'lm_vector_arithmetic'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 35 (delta 10), reused 21 (delta 4), pack-reused 0
Receiving objects: 100% (35/35), 236.07 KiB | 8.14 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [2]:
cd lm_vector_arithmetic

/content/lm_vector_arithmetic


In [3]:
%%capture
!pip install -r requirements.txt

In [4]:
# !pip install transformers

# import torch
# import transformers
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import matplotlib.pyplot as plt

# def load_gpt2(version):
#     device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
#     tokenizer = AutoTokenizer.from_pretrained(version)
#     model = AutoModelForCausalLM.from_pretrained(version, torch_dtype=torch.float16).to(device)
#     return model, tokenizer

In [5]:
from modeling import *  #GPT2Wrapper, ModelWrapper

In [6]:
model, tokenizer = load_gpt2('gpt2')
model = model.float()
wrapper = GPT2Wrapper(model, tokenizer)

(…)ingface.co/gpt2/resolve/main/config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

(…)gingface.co/gpt2/resolve/main/vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

(…)gingface.co/gpt2/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)face.co/gpt2/resolve/main/tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/548M [00:00<?, ?B/s]

# Functions

In [7]:
def tokenize(text):
    inp_ids = wrapper.tokenize(text)
    str_toks = wrapper.list_decode(inp_ids[0])
    return inp_ids, str_toks

def get_tokIDS(test_text):
    test_ids, test_toks = tokenize(test_text)
    logits = wrapper.get_layers(test_ids)
    wrapper.print_top(logits[1:]) #skip the first embedding layer b/c it just reverses to input
    return test_ids

In [8]:
def create_layer_list(X, Y):
    layer_list = []
    for layer in range(X, Y+1):
        layer_list.append("Layer {}".format(layer))
    return layer_list

from tabulate import tabulate
def create_table(list1, list2):
    table_data = zip(list1, list2)
    headers = ["Layer", "Orig Top Token"]
    table = tabulate(table_data, headers, tablefmt="grid")
    return table

import torch.nn.functional as F
def get_decoded(logits, k=10):
    output_list = []
    for i,layer in enumerate(logits):
        output_list.append( wrapper.tokenizer.decode(F.softmax(layer,dim=-1).argsort(descending=True)[:k]) )
    return output_list

def format_to_table(tok_ids):
    out = wrapper.model(input_ids = tok_ids, output_hidden_states=True)
    logits = out.logits
    hidden_states = out.hidden_states
    hidden_states = list(hidden_states)[1:]

    orig_layer_logits = wrapper.layer_decode(hidden_states)
    orig_layer_logits = torch.stack(orig_layer_logits).squeeze(-1)

    orig_decoded = get_decoded(orig_layer_logits)
    table_output = create_table(create_layer_list(0, 23), orig_decoded)
    print(table_output)

# Test Incr Digits

In [ ]:
test_text = """1 2 3 4"""
test_ids = get_tokIDS(test_text)

0 thteenthx 4 3teen3454 239
1 thteenth54x3934teen384174
2 thx3454GHz388639teen74
3 3454thGHz86ts743979x
4 tsms34thths54 -iel86 4
5 ts 4 - 3 5ths 6 +ms34
6  4★ 3 5 6 2 Tycoon >>> +ts
7  4 5 6 3 0 2 Tycoon 1ths 8
8  4 5 3 6 1 2 8 7★ 9
9  5 4 6 3 75 1 9 0★
10  5 6 4 3 7 1 05 9/
11  5 4 1 6 3 0/ 7
5


In [ ]:
format_to_table(test_ids)

+----------+---------------------------+
| Layer    | Orig Top Token            |
+==========+===========================+
| Layer 0  | thteenthx 4 3teen3454 239 |
+----------+---------------------------+
| Layer 1  | thteenth54x3934teen384174 |
+----------+---------------------------+
| Layer 2  | thx3454GHz388639teen74    |
+----------+---------------------------+
| Layer 3  | 3454thGHz86ts743979x      |
+----------+---------------------------+
| Layer 4  | tsms34thths54 -iel86 4    |
+----------+---------------------------+
| Layer 5  | ts 4 - 3 5ths 6 +ms34     |
+----------+---------------------------+
| Layer 6  | 4★ 3 5 6 2 Tycoon >>> +ts |
+----------+---------------------------+
| Layer 7  | 4 5 6 3 0 2 Tycoon 1ths 8 |
+----------+---------------------------+
| Layer 8  | 4 5 3 6 1 2 8 7★ 9        |
+----------+---------------------------+
| Layer 9  | 5 4 6 3 75 1 9 0★         |
+----------+---------------------------+
| Layer 10 | 5 6 4 3 7 1 05 9/         |
+----------+----

In [15]:
test_text = """11 12 13"""
test_ids = get_tokIDS(test_text)

0 th13153714 113311 13.
1 th371566451373331476
2 37th6645333426401544
3 3733663445th86762615
4 3745346644th86763346
5  - /. � winds left:th37 0
6  0 /: � 11 -. 9 00 winds
7  0 13 11 9 14 # 00: 3 7
8  13 0 14 9 11 12 7 313 8
9  14 1314 12 15 16 013 4 18
10  14 13 12 15 16 0 18 8 4 11
11  14: 13 15 12. 16 0 4 1


In [19]:
test_text = """1 2 3 4 5 6 7 8 9 10 11"""
test_ids = get_tokIDS(test_text)

0  11th1011403045311236
1 th453150405530 113710
2 .36,34373830314550
3 .373034453638555019
4 .45 Air 9, >>>3455 2436
5 . | 9 >>> ],work directions 24 13
6  11
 9 >>>. ] 12 13 6,
7  11 12 9 13 41 N 7
 1 6
8  11 12 10 13 11111 9 # Q 2011
9  1212 11 13 2012 root 22 112 14 42
10  1212 13 14 2012 11 8 22 # 0
11  1212 2012
 # | 2 \ 13 0


In [20]:
test_text = """1 2 3 4 5 6 7 8 9 10 11 12"""
test_ids = get_tokIDS(test_text)

0 34314520151312111636
1 45345031513620378013
2 3445365020313837.60
3 344550.3620371338 Apostles
4 3445 Deck Apostles362050 Date80.
5  Date34 seasons 13 9 Hook ]45. 14
6  12 13 //[ ]
 9 Hook Deck 14 2012
7  12 13 //[ 14 11 2012 24 16 Ruler 9
8  12 13 11 14 2012 #12 Rath13 9
9  1313 14 2013 12 123 23 63 3 XIII
10  1313 14 3 2013 12 23 # 123 0
11  13 3 #13 2013 14
 0 1 t


In [23]:
test_text = """1 2 3 4 5 6 7 8 9 10 11 12 13 14"""
test_ids = get_tokIDS(test_text)

0 th3138373415144036teenth
1 th40.54383655373134
2 .th,363438:375440
3 .363438375530 30th39
4  winds. ends Date3436 left Link Team seasons
5  Date winds seasons.-| 14 | 24 ends -
6 -| 14
 | winds 9 13;; 24 <!--
7  14 16-| <!-- 15 13 2014 24
 //[
8  14 15 13 16 #14 2014 12 1 17
9  1515 14 16 2015 145 17 150 <!-- 13
10  15 16 1415 17 2015 # 13 150 1
11  1515 # 2015 5
 < 1 16 25


# Test months

In [ ]:
test_text = """January February March April"""
test_ids = get_tokIDS(test_text)

0  2015 May 2014 April June Aug 2013 July 2017 2016
1  2015 2014 2017 2013 2016 29 2018 27 May 24
2  2015 2014 29 2017 28 27 30 2013 2018 2016
3  29 2015 27 28 30 24 2014 25 26 23
4  24 29 2014 2015 2018 2017 27 30 28 26
5  29 24 30 28 27 4 3 26 2014 6
6  29 24 30 2014 28 26 2018 31 27 2015
7  2014 29 April 24 2015 2018 2019 2013 4 28
8  April 2014 May March 2015 June September 2017 October Apr
9  May April June 2015 5 July 2005 September March November
10  May June July AprilMay November September 5 March October
11  May June July 5
 April - MarchMay November


In [ ]:
format_to_table(test_ids)

+----------+-----------------------------------------------------------+
| Layer    | Orig Top Token                                            |
+==========+===========================================================+
| Layer 0  | 2015 May 2014 April June Aug 2013 July 2017 2016          |
+----------+-----------------------------------------------------------+
| Layer 1  | 2015 2014 2017 2013 2016 29 2018 27 May 24                |
+----------+-----------------------------------------------------------+
| Layer 2  | 2015 2014 29 2017 28 27 30 2013 2018 2016                 |
+----------+-----------------------------------------------------------+
| Layer 3  | 29 2015 27 28 30 24 2014 25 26 23                         |
+----------+-----------------------------------------------------------+
| Layer 4  | 24 29 2014 2015 2018 2017 27 30 28 26                     |
+----------+-----------------------------------------------------------+
| Layer 5  | 29 24 30 28 27 4 3 26 2014 6          

# Test Decr Digits

In [9]:
test_text = """21 20 19 18 17 16 15 14 13 12"""
test_ids = get_tokIDS(test_text)

0 4534131112153120 1114
1 453450th513156206036
2 34453638502031374060
3 34453638503731512033
4 45345051566036.5490
5 4534. -3656 9 Command799999
6 34.45 116465636 9 12 3
7  0 12 11 6 13 3 9 4 14 2
8  13 12 11 14 0 9 3 6 7 8
9  11 12 13 9 119 10 3 0 123 14
10  11 12 10 9 13 7 09 811 0
11  11 12 10 9 13 7 8 09
 14


In [10]:
format_to_table(test_ids)

+----------+------------------------------+
| Layer    | Orig Top Token               |
+==========+==============================+
| Layer 0  | 4534131112153120 1114        |
+----------+------------------------------+
| Layer 1  | 453450th513156206036         |
+----------+------------------------------+
| Layer 2  | 34453638502031374060         |
+----------+------------------------------+
| Layer 3  | 34453638503731512033         |
+----------+------------------------------+
| Layer 4  | 45345051566036.5490          |
+----------+------------------------------+
| Layer 5  | 4534. -3656 9 Command799999  |
+----------+------------------------------+
| Layer 6  | 34.45 116465636 9 12 3       |
+----------+------------------------------+
| Layer 7  | 0 12 11 6 13 3 9 4 14 2      |
+----------+------------------------------+
| Layer 8  | 13 12 11 14 0 9 3 6 7 8      |
+----------+------------------------------+
| Layer 9  | 11 12 13 9 119 10 3 0 123 14 |
+----------+--------------------

In [11]:
test_text = """5 4 3 2"""
test_ids = get_tokIDS(test_text)

0 nd 2 3x 1 4200 -db
1 ndx 2 3½200 1 -db
2 ndxnder -200ts 2 3 1½
3 ndndertsx - 3200amins133GHz
4 ndnderts - 3x 4. 2 6
5 nd -ts 3 2x + 4 1 6
6 nd½ 2 3 - 1 4 + 0.
7 nd 2 3 0 1 4 6 5 - +
8  3 2 0 4 1 5 6nd 7 8
9  2 3 0 1 4 5 6 7 8 9
10  0 1 2 3 4 5 6 7 9 8
11  1 2 0 3 4 5. 6 7 -


In [12]:
test_text = """6 5 4 3"""
test_ids = get_tokIDS(test_text)

0 rdx 3d½ 2 1th-c
1 rdxdth 3½200.RD�
2 rdxd 3 -200nderminXxx
3 rdx -tsnder 3aminsd windsmin
4 rdx -ts 3nder 6. glassesmin
5 rd - 3xts + 6 4 2.
6 rd 3x - 2 4 1 + 6½
7  3rd 4 0 2 1 6 5 7 -
8  3 4 2 0 5 1 6 7 8 9
9  3 4 2 0 1 6 5 8 7 9
10  2 4 3 0 1 6 5 8 7 9
11  2 3 1 4 0 5. 6 8 7


In [13]:
test_text = """10 9 8 7 6"""
test_ids = get_tokIDS(test_text)

0 th363134383921242664
1 th363138393464512221
2 th3634383139x265445
3 3436313938th54797426
4 thts 634ths -xminteen79
5  -x 63634 inchesth79. 9
6  6x. - 434 536th 7
7  6 4 7 5 3 0 9 1 8 2
8  6 7 5 4 3 8 0 9 2 1
9  5 6 7 3 4 0 2 1 8 9
10  5 6 7 4 3 0 2 1 8 9
11  5 6 7 4 3 1 2 8 0.


In [14]:
test_text = """10 9 8 7"""
test_ids = get_tokIDS(test_text)

0 th161114211739386915
1 th693839312137433617
2 th383431463969362635
3 th346979863126743946
4 th - 9 779 68669tsx
5  - 7 9th. 6799999x 8
6  7 - 6 9.x 8 4 3 5
7  7 6 9 8 4 0 3 5 1 -
8  7 6 8 9 0 5 4 3 1 2
9  7 6 8 5 4 0 9 3 2 1
10  7 6 8 9 5 4 0 2 3 1
11  7 6 8 5 4 9 0. 2 1


In [26]:
test_text = """15 14 13 12 11"""
test_ids = get_tokIDS(test_text)

0  11th111012374540 1277
1 th4530 11405010315541
2 34th38364530314037.
3 343645303831378639th
4 . 9 6th 764645 3 4 5
5  9. 6 7 - 3 11 4 8 Trib
6 . 9 11 6 - 3 7 4646,
7  11 6 9 0 7 4 3 5 8 -
8  11 7 0 6 9 4 3 13 8 5
9  11 10 12 5 9 0 6 4 8 7
10  11 10 12 9 8 0 7 13 14 6
11  10 11 9 12 8 7 6 5. 13


In [27]:
test_text = """16 15 14 13 12 11"""
test_ids = get_tokIDS(test_text)

0  11th11103712 12454030
1 th45304050 1110314155
2 3438th3630453140.37
3 343638453731303986.
4 . 9 6 764645th 5 4 3
5  9. 6 7 11 3 - 4 8 10
6 . 11 9 6646 3 7 - 436
7  11 6 9 4 3 7 0 5 8 1
8  11 6 7 9 0 4 12 3 13 8
9  11 10 5 12 9 4 8 0 6 7
10  10 11 12 9 8 0 5 7 6 15
11  10 11 9 8 12 5 7 6. 4


In [29]:
test_text = """18 17 16 15 14 13 12 11"""
test_ids = get_tokIDS(test_text)

0  11th111037 12 13404531
1 th4045 11503031104137
2 3834th36.3031403745
3 3834363745.31303988
4 . 9 6 745 438 5 8 18
5  9. 6 7 3 4 11 - 5work
6 . 9 11 4 7 6 3, 8 -
7  11 6 9 4 7 3 0 5 8 1
8  11 12 6 7 9 13 4 0 10 8
9  10 11 12 5 9 8 4 0 6 Ten
10  10 11 12 8 9 0 5 09 7 6
11  10 11 9 8 12 5 7
 100 6


In [30]:
test_text = """21 20 19 18 17 16 15 14"""
test_ids = get_tokIDS(test_text)

0 th141537 14384034.54
1 th.4038544586373436
2 .thmin3438,:373640
3 343837min313654.7986
4 .9999 Command79th:,8654min
5 .999979 - Command, inches:.34
6 .9999: 14.8679, Team31
7  14 0 1314 16 9 6 7 15 4
8  14 13 15 16 0 17 12 7 11 9
9  13 14 15 12 11 013 714 16
10  13 14 15 12 11 17 16 10 7 9
11  13 14 12 15 11 10 7 9 17 16


In [31]:
test_text = """100 99 98 97"""
test_ids = get_tokIDS(test_text)

0  97 96 99 98 95 94 7690 9380
1 80603090704075995045
2 3080996075%999904045
3 609980%9997545653040
4 999999999%60807565603807
5 999999999% 99ers.8075 �
6 %999999 99 98 96 97999 95 81
7  97 98 96 87 95 99 77% 81 89
8  97 98 96 99 95 87 77 88 67 93
9  98 96 97 99 88 95 94 929798
10  98 96 97 99 94 95 92 88 86 100
11  98 96 97 100 99 95. 94 92 88


In [32]:
test_text = """100 99 98 97 96"""
test_ids = get_tokIDS(test_text)

0  96809060 72 94th3064 66
1 8060306445709050th 96
2 80306050454036759070
3 80609999303645994090999
4 9999999806099%. mphts 6
5 9999999 9699.80 mph% 99 55
6  969999%8099uzz 76 95 56 86
7  96 95 98 97 0 94 76 1996 86%
8  96 98 97 95 99 94 79 87 76 66
9  95 96 97 93 98 94 99 8797 92
10  95 97 96 93 99 94 98 92 87 91
11  95 97 96 93 99
. 100 94 90


In [33]:
test_text = """99 98 97 96 95"""
test_ids = get_tokIDS(test_text)

0  9590% 94 90 96 8095 9999
1  9590% 94459975609540
2 % 959990%:75 94453060
3  95%999999%:7590 996045
4 9999%:% 9599999 mph 100 99.
5 9999 95% 99 96.999 mph 94 100
6  95% 969999 99 94 85%: mph 65
7  95 96 94% 93 85 97 98 91 87
8  95 96 97 98 94 99 93 87 85 86
9  94 96 95 93 98 92 86 97 91 88
10  94 96 92 95 93 98 91 86 88 89
11  94 96 95 93 92 98 91 90 89 86
